# Relatório Parcial - Maratona de Filmes

* Matéria: Supercomputação
* Aluno: Guilherme Lunetta

## O problema

O objetivo deste relatório é apresentar a solução para o desafio de selecionar o maior número possível de filmes para serem assistidos em um dia, levando em consideração as restrições de horários disponíveis e o número máximo de filmes permitidos em cada categoria. 

A entrada consiste em um inteiro N, representando o número de filmes disponíveis, e N trios de inteiros (H[i], F[i], C[i]), representando a hora de início, a hora de fim e a categoria do i-ésimo filme. Além disso, há um inteiro M, representando o número de categorias, e uma lista de M inteiros representando o número máximo de filmes que podem ser assistidos em cada categoria. A saída esperada é um inteiro representando o número máximo de filmes que podem ser assistidos de acordo com as restrições de horários e número máximo por categoria, a porcentagem de tempo de tela e os filmes escolhidos.

Além disso, devemos desenvolver duas soluções distintas, uma solução usando uma heurística gulosa e a outra usando a mesma heurística gulosa porém com uma aleatorização, buscando novas soluções.

**Exemplo de arquivo de entrada:**
```txt
10 4
1 3 1 2 
11 13 3
14 15 3
10 16 2
10 14 1
11 17 2
11 14 3
13 15 3
14 15 1
12 16 4
12 13 4
```

**Exemplo de arquivo de saída:**
```txt
2
12.5
10 14 1
14 15 3
```

## Implementação

A implementação foi feita da seguinte forma:

* **1)** Leitura do arquivo de entrada e armazenamento em vetores, estruturas customizadas, constantes e variáveis.

* **2)** Ordenar os filmes por hora de término, do menor para o maior. Caso os horários de término sejam iguais, ordenamos por hora de início, da menor para a maior.

* **3)** Escolher os filmes que serão assistidos, seguindo a regra de que um filme não pode começar ou terminar durante a exibição de outro filme já escolhido e a regra de que um filme não pode ser escolhido se o máximo de filmes de sua categoria já foi atingido.
    
    * 3.1) Também foram ignorados os filmes que começam e terminam no mesmo horário, visto que é impossível assistir um filme assim.
    
    * 3.2) Por fim, foram ignorados filmes que começam depois de terminar (confuso, né?) pois isso é impossível sem uma máquina do tempo.

* **4)** Na heurística gulosa, escolhe-se o filme que termina mais cedo, desde que respeite a regra da categoria e a regra de exibição. Na heurística gulosa aleatorizada, existe uma chance de 25% onde escolhe-se um filme aleatório da lista de filmes, desde que respeito a regra da categoria e da exibição e os outros 75% de chance, segue a heurística gulosa padrão.

* **5)** Escrever o arquivo de saída contendo o número de filmes escolhidos, a porcentagem de tempo de tela e a lista de filmes escolhidos com suas informações, em ordem.

## Heurística Gulosa

Os filmes foram transformados em uma estrutura de dados com as suas informações:

```c++
struct movie {
    int index;
    int start;
    int end;
    int category;
};
```

Para saber quais horários do dia ainda estavam disponíves a fim de escolher se um filme será assistido ou não, foi necessário usar uma estrutura de dados chamada **bitset**. No entanto, foi desenvolvida uma estrutura **bitset** customizada para o problema, diferente da disponível na biblioteca **bitset** do C++. Segue o código abaixo:

```c++
struct rbitset {
    int size = 24;
    vector<int> bs = vector<int>(24, 0);

    int get(int index) {
        return bs[index];
    }

    void set(int index) {
        bs[index] = 1;
    }

    void print() {
        for (int i = 0; i < size; i++) {
            cout << bs[i];
        }

        cout << endl;
    }
};
```

Como dito anteriormente, a função que ordena os filmes por hora de término, da menor para a maior também os ordena por hora de início, da menor para a maior, caso existam filmes em que a hora de término seja a mesma, melhorando ainda mais as escolhas. Segue o código abaixo da função:

```c++
bool orderByEndTime(movie &a, movie &b) {
    if (a.end != b.end) {
        return a.end < b.end;
    }

    return a.start < b.start;
}
```

Seguindo nosso algoritmo, chegamos na parte de escolher os filmes, isso foi feito em duas partes, a primeira é uma função que diz se o filme é elegível para assistir seguindo as regras mencionadas anteriormente. E a segunda é o loop principal do algoritmo que preenche o vetor de filmes escolhidos e atualiza o vetor de filmes por categoria caso o filme seja elegível. Segue a função e o loop principal abaixo:

```c++
bool isWatchable(int start, int end, rbitset &bitset) {
    if (start == end) {
        return false;
    }

    if (start > end) {
        return false;
    }

    for (int i = start; i < end; i++) {
        if (bitset.get(i)) {
            return false;
        }
    }

    return true;
}

for (int i = 0; i < numMovies; i++) {
    if (categories[movies[i].category] != 0) {
        if (isWatchable(movies[i].start, movies[i].end, bitset)) {
            populateBiset(movies[i].start, movies[i].end, bitset);
            chosenMovies.push_back(movies[i]);
            categories[movies[i].category]--;
        }
    }
}
```

Lembrando que a função **populateBitset()** apenas atualiza o bitset com os horários do novo filme para que não ocorra nenhum equívoco na escolha dos próximos filmes.

Por fim, a função de escrita do arquivo de output que, como dito anteriormente, cria um arquivo e coloca as informações importantes, sendo elas: número de filmes escolhidos, porcentagem de tempo de tela e os filmes escolhidos em ordem. Segue o código abaixo: 

```c++
void writeOutput(string filename, vector<movie> movies) {
    int numMovies = movies.size();

    int hoursWithoutMovies = 0;
    for (int i = 0; i < movies.size()-1; i++) {
        hoursWithoutMovies += movies[i+1].start - movies[i].end;
    }

    if (movies[numMovies-1].end != 24) {
        hoursWithoutMovies += 24 - movies[numMovies-1].end;
    }

    if (movies[0].start != 0) {
        hoursWithoutMovies += movies[0].start;
    }

    double screenPercentage = ((24 - hoursWithoutMovies) * 100) / 24.0;

    fs::path dir_path("outputs/");
    if (!fs::exists(dir_path)) {
        fs::create_directory(dir_path);
    }

    ofstream inputFile;
    filename = filename.replace(0, 5, "outputs/output-gulosa");
    inputFile.open(filename);
    inputFile << numMovies << endl;
    inputFile << screenPercentage << endl;

    for (int i = 0; i < movies.size(); i++) {
        inputFile << movies[i].index << " " << movies[i].start << " " << movies[i].end << " " << movies[i].category << endl;
    }

    inputFile.close();
}
```

## Heurística Gulosa Aleatória

Na heurística gulosa aleatorizada, a única diferença é no loop principal do algoritmo, onde antes de checarmos se um filme é elegível ou não, checamos se esse é um caso dos 25% de chance de escolher um filme aleatório, se sim, fazemos as mesmas checagens que a heurística gulosa padrão e continuamos. Segue o código novo abaixo: 

```c++
for (int i = 0; i < numMovies; i++) {
    if (shouldChoose(0.25, dgenerator, ddistribution)) {
        int idx = randomInt(igenerator, idistribution);
        if (categories[movies[idx].category] != 0) {
            if (isWatchable(movies[idx].start, movies[idx].end, bitset)) {
                populateBiset(movies[idx].start, movies[idx].end, bitset);
                chosenMovies.push_back(movies[idx]);
                categories[movies[idx].category]--;
            } 
        }
    } else {
        if (categories[movies[i].category] != 0) {
            if (isWatchable(movies[i].start, movies[i].end, bitset)) {
                populateBiset(movies[i].start, movies[i].end, bitset);
                chosenMovies.push_back(movies[i]);
                categories[movies[i].category]--;
            }
        }
    }
}
```

Também temos a função que nos diz se devemos escolher um filme aleatório ou não baseado na chance de 25% citada anteriormente. Segue o código abaixo:

```c++
bool shouldChoose(double prob, default_random_engine &eng, uniform_real_distribution<double> &dist) {
    if (dist(eng) <= prob) {
        return true;
    }

    return false;
}
```

## Profiling

Utilizando o *valgrind*, iremos analisar o comportamento de cada algoritmo, buscando entender onde o algoritmo tem mais chamadas e pode ser otimizado. Ambas análises foram feitas utilizando o input com 100000 filmes e 20 categorias. 

### Profiling do algoritmo guloso

No algoritmo guloso, 57,86% das chamadas do algoritmo estão no loop de leitura dos filmes do arquivo de input e 37,04% das chamadas estão na função **sort()** de ordenação, como mostra a imagem abaixo.

OBS: Não coloquei o resultado inteiro do valgrind pois este tem aproximadamente 6 mil linhas.

<img src="valgrind/gulosa.png" height="400">

### Profiling do algoritmo guloso aleatório

Assim como no algoritmo guloso, 54,55% das chamadas do algoritmo estão no loop de leitura dos filmes do arquivo de input e 34,82% das chamadas estão na função **sort()** de ordenação, como mostra a imagem abaixo.

<img src="valgrind/aleatoria.png" height="400">

### Conclusão do profiling

Conclui-se que o loop de leitura dos filmes e a função **sort()** são os lugares do código que necessitam de atenção quanto ao desempenho e podem ser alvos de pesquisa para otimização.

## Comparação entre as heurísticas

Vamos comparar os resultados de cada heurística. Vamos comparar o número de filmes escolhidos, o tempo de execução de cada algoritmo variando os inputs e a porcentagem de tempo de tela. Os resultados serão mostrados por tabelas e gráficos.

### Número de filmes escolhidos

In [ ]:
# Importando todas as bibliotecas necessárias

In [ ]:
# Abrindo arquivo csv em um DataFrame do pandas

In [ ]:
# Código para analise do número de filmes escolhidos

### Tempo de execução

In [ ]:
# Código para analisar tempo de execução entre algoritmos

### Porcentagem de tempo de tela

In [ ]:
# Código para analisar porcentagem de tempo de tela